**1. Self Attention📌**

아래 코드를 수행해보고, Self Attention은 어떤 메커니즘인지 설명하고,

 그 설명에 맞게 각 코드에 직접 주석을 달아봅시다.

 ✅설명:   Attention중에서 자기 자신에게 Attention 메커니즘을 행하는 방식. (Query = Key = Value)

In [1]:
import numpy as np

# 간단한 문장: ['나는', '사과를', '먹었다']
words = ['나는', '사과를', '먹었다']

# word 를 vector 로 embedding
word_vectors = {
    '나는': np.array([1, 0, 0]),
    '사과를': np.array([0, 1, 0]),
    '먹었다': np.array([0, 0, 1])
}

# 유사도 계산 함수 (self-attention)
def self_attention(query, key):
    return np.dot(query, key) # query 와 key 내적 (둘 사이 연관성 계산 위해서)

attention_matrix = np.zeros((len(words), len(words))) # 위의 내적 값이 attention score 로 저장하기 위해 동일 크기의 0 벡터 생성
for i in range(len(words)):
    for j in range(len(words)):
        attention_matrix[i][j] = self_attention(word_vectors[words[i]], word_vectors[words[j]]) # 각 벡터간 얼마나 유사한지 내적 계산(자기자신과 계산 경우도 있음)

print("Self-Attention Matrix:")
print(attention_matrix)


Self-Attention Matrix:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


**2. Multi-Head Self Attention📌**

아래 코드를 수행해보고, Multi-Head Self Attention은 어떤 메커니즘인지 설명하고,

 그 설명에 맞게 각 코드에 직접 주석을 달아봅시다.

 ✅설명: Query, Key, Value값을 한 번에 계산하지 않고 head 수만큼 나눠 계산 후 나중에 Attention Value들을 합치는 메커니즘. 한마디로 분할 계산 후 합산하는 방식.

Multi-head Attention

1. 원래 Query, Key, Value 행렬 값을 head 수만큼 분할

2. 분할된 행렬 값을 통해, 각 Attention value값들을 도출

3. 도출된 Attention value값들을 concatenate(쌓아 합치기)하여 최종 Attention value도출

In [5]:
# 여러 개의 attention heads
def multi_head_self_attention(query, key, heads=3):
    return [np.dot(query, key) for _ in range(heads)]  # 각 헤드에 대해서 attention 계산

multi_head_attention_matrix = np.zeros((len(words), len(words), 3)) # head 3개 이므로 attention value 저장할 0 매트리스 생성
for i in range(len(words)):
    for j in range(len(words)):
        multi_head_attention_matrix[i][j] = multi_head_self_attention(word_vectors[words[i]], word_vectors[words[j]])

print("\nMulti-Head Self-Attention Matrix:")
print(multi_head_attention_matrix)



Multi-Head Self-Attention Matrix:
[[[1. 1. 1.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [1. 1. 1.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [1. 1. 1.]]]


**3. Masked Multi-Head Self Attention📌**

아래 코드를 수행해보고, Masked Multi-Head Self Attention은 어떤 메커니즘인지 설명하고,

 그 설명에 맞게 각 코드에 직접 주석을 달아봅시다.

 ✅설명: 디코더(Decoder)에서 사용되는 기법으로, 다중 헤드(Self-Attention)의 변형입니다. 주요 목표는 미래 정보를 차단하여, 모델이 시퀀스를 예측할 때 현재 시점 이후의 단어들을 보지 못하게 하는 것이고, 메커니즘은
1. Query, Key, Value 벡터 생성 후,

2. 미래 단어들에 대한 마스킹 진행되는 데 마스킹 행렬은 상삼각 행렬(Upper Triangular Matrix) 형태로 이렇게 하면 각 단어가 자신을 포함해 그 이전 단어들에만 Attention을 할 수 있고, 이후의 단어는 무시됩니다.
3. Attention Score 계산 및 마스킹 적용: 마스크 행렬에 0 대신 매우 큰 음수를 사용하여 소프트맥스 함수에 의해 해당 가중치가 0으로 되도록 만듭니다. 즉, 미래 단어에 대한 가중치는 0이 되고, 모델은 오직 자신과 그 이전 단어들에만 집중하게 됩니다.


In [6]:
# 마스크 추가: 현재 단어 이후의 단어는 계산하지 않음
def masked_attention(query, key, mask):
    return np.dot(query, key) * mask #

mask = np.array([1, 1, 0])  # 첫 번째, 두 번째는 보지만, 세 번째는 Mask (이 빈칸 꼭 채워주세요 :) )
masked_attention_matrix = np.zeros((len(words), len(words)))
for i in range(len(words)):
    for j in range(len(words)):
        masked_attention_matrix[i][j] = masked_attention(word_vectors[words[i]], word_vectors[words[j]], mask[j])

print("\nMasked Self-Attention Matrix:")
print(masked_attention_matrix)



Masked Self-Attention Matrix:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]


**4. Cross Attention📌**

아래 코드를 수행해보고, Cross Attention은 어떤 메커니즘인지 설명하고,

 그 설명에 맞게 각 코드에 직접 주석을 달아봅시다.

 ✅설명: 디코더의 Query 벡터가 인코더의 Key 및 Value 벡터와 상호작용하는 방식입니다. 즉, 디코더는 자기 자신에 대한 정보뿐만 아니라 인코더가 생성한 문맥 정보를 활용하여 다음 단어를 예측합니다.

In [8]:
# 입력 문장과 응답 문장
question_words = ['너는', '사과를']
answer_words = ['나는', '먹었다']
question_vectors = {
    '너는': np.array([1, 0]),
    '사과를': np.array([0, 1])
}
answer_vectors = {
    '나는': np.array([1, 0]),
    '먹었다': np.array([0, 1])
}

# Cross-Attention
cross_attention_matrix = np.zeros((len(question_words), len(answer_words)))
for i in range(len(question_words)):
    for j in range(len(answer_words)):
        cross_attention_matrix[i][j] = np.dot(question_vectors[question_words[i]], answer_vectors[answer_words[j]]) # question 과 answer 이 상호작용

print("\nCross-Attention Matrix:")
print(cross_attention_matrix)


Cross-Attention Matrix:
[[1. 0.]
 [0. 1.]]
